In [2]:
import os
from typing import List, Dict, Any
import pandas as pd

### LangChain Document Structure

In [3]:
from langchain_core.documents import Document

In [4]:
doc = Document(
    page_content="This is some sample content loaded from example.txt.",
    metadata={
        "source": "example.txt",
        "author": "Tarique",
        "page": 1,
        "created_date": "2025-12-25"
    }
)
print("Document Structure is as follows")
print(f"Page Content: {doc.page_content}")
print(f"Meta Data: {doc.metadata}")

# Why metadata:
print("\nMetadata is crucial for:")
print("- Filtering search results")
print("- Tracking document sources")
print("- Providing context in responses")
print("- Debugging and auditing")

Document Structure is as follows
Page Content: This is some sample content loaded from example.txt.
Meta Data: {'source': 'example.txt', 'author': 'Tarique', 'page': 1, 'created_date': '2025-12-25'}

Metadata is crucial for:
- Filtering search results
- Tracking document sources
- Providing context in responses
- Debugging and auditing


In [5]:
type(doc)

langchain_core.documents.base.Document

### Reading a .txt file

In [6]:
## Create a text file
os.makedirs("data/text_files", exist_ok=True)

In [7]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",

"data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)
print("Sample text files created")

Sample text files created


### TextLoader- Read Single File 

In [8]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path="data/text_files/python_intro.txt", encoding="utf-8")
documents = loader.load()

print(type(documents), type(documents[0]))
print(f"Loaded {len(documents)} documents")
print(f"Page content: {documents[0].page_content[:160]}")
print(f"Metadata: {documents[0].metadata}")

/Users/apple/Desktop/AI Practice Projects/RAG_Bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'list'> <class 'langchain_core.documents.base.Document'>
Loaded 1 documents
Page content: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum 
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader- Multiple Text Files

In [9]:
from langchain_community.document_loaders import DirectoryLoader

directory_loader = DirectoryLoader(
    path="data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'},
    show_progress=True
)

documents = directory_loader.load()
print(f"Loaded {len(documents)} documents")
print(type(documents), type(documents[0]))

for i, doc in enumerate(documents):
    print(f"{i+1} Document: Source-{doc.metadata['source']} Length-{len(doc.page_content)}")

100%|██████████| 2/2 [00:00<00:00, 1876.23it/s]

Loaded 2 documents
<class 'list'> <class 'langchain_core.documents.base.Document'>
1 Document: Source-data/text_files/python_intro.txt Length-489
2 Document: Source-data/text_files/machine_learning.txt Length-575


### Text Splitting Stratergies

In [10]:
documents

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'),
 Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Lear

###  1. CharacterTextSplitter

In [11]:

from langchain_text_splitters import CharacterTextSplitter

text = documents[0].page_content

char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

char_chunks = char_splitter.split_text(text)

print(f"Splitted into {len(char_chunks)} chunks")
print("*********************************************")
print(char_chunks[0])
print("*********************************************")
print(char_chunks[1])
print("*********************************************")
print(char_chunks[2])

Splitted into 3 chunks
*********************************************
Python Programming Introduction
Python is a high-level, interpreted programming language known for its simplicity and readability.
*********************************************
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.
Key Features:
- Easy to learn and use
- Extensive standard library
*********************************************
- Cross-platform compatibility
- Strong community support
Python is widely used in web development, data science, artificial intelligence, and automation.


### 2. RecursiveCharacterTextSplitter

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_char_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

r_chunks = recursive_char_splitter.split_text(text)

print(f"Splitted into {len(r_chunks)} chunks")
print("*********************************************")
print(r_chunks[0])
print("*********************************************")
print(r_chunks[1])
print("*********************************************")
print(r_chunks[2])


Splitted into 5 chunks
*********************************************
Python Programming Introduction
*********************************************
Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
*********************************************
programming languages in the world.


### Chunk overlap 

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text without natural break points
simple_text_chars = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],  # Only split on spaces
    chunk_size=80,
    chunk_overlap=20,
    
    length_function=len
)

simple_chunks = splitter.split_text(simple_text_chars)
print(f"Splitted into {len(simple_chunks)} chunks")

for i, chunk in enumerate(simple_chunks):
    print(f"Chunk {i+1}: {chunk}")


Splitted into 4 chunks
Chunk 1: This is sentence one and it is quite long. This is sentence two and it is also
Chunk 2: two and it is also quite long. This is sentence three which is even longer than
Chunk 3: is even longer than the others. This is sentence four. This is sentence five.
Chunk 4: is sentence five. This is sentence six.


### 3.TokenTextSplitter

In [20]:
from langchain_text_splitters import TokenTextSplitter


token_splitter = TokenTextSplitter(
    chunk_size=50, # size in tokens not characters
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(text)
print(f"Split into {len(token_chunks)} chunks")

for i, chunk in enumerate(token_chunks):
    print(f"{i+1} chunk: {chunk}")
    print("*********************************************")

Split into 3 chunks
1 chunk: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in
*********************************************
2 chunk:  one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web
*********************************************
3 chunk:  community support

Python is widely used in web development, data science, artificial intelligence, and automation.
*********************************************
